In [122]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [123]:
data = pd.read_csv('clean_steam_data.csv')
# print(data.shape)
# data = data[data['Reviews'] > 0]
# print(data.shape)
data.drop('Name', axis=1, inplace=True)

# scaler = StandardScaler()
# columns = data.columns
# data = pd.DataFrame( scaler.fit_transform(data), columns=columns)



train, test = train_test_split(data, train_size=0.75,test_size= 0.25,random_state=123 )

y_train = train['Reviews'].copy()
y_test  = test['Reviews'].copy()

train.drop(["Reviews"], axis=1, inplace=True) #,'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos'
test.drop( ["Reviews"], axis=1, inplace=True) #,'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos'
# scaler = StandardScaler()

# train = scaler.fit_transform(train)
x_train  = train.copy()
x_test   = test.copy()

In [124]:
# import numpy as np
# ytrain = np.array(y_train)
# ytrain = ytrain.reshape(-1, 1)
# ytrain = scaler.fit_transform(y_train)
ytrain = pd.DataFrame(y_train)
ytrain.describe()

,Reviews
count,6474.000000
mean,2236.479302
std,3817.687066
min,22.000000
25%,347.000000
50%,781.000000
75%,2227.500000
max,34054.000000


In [125]:
# inputs = tf.keras.Input(shape=(91,))
# x = tf.keras.layers.Dense(128, activation='relu')(inputs)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# outputs = tf.keras.layers.Dense(1)(x)

# model = tf.keras.Model(inputs=inputs, outputs=outputs)


# optimizer = tf.keras.optimizers.RMSprop(0.001)

# model.compile(
#     optimizer=optimizer,
#     loss='mse'
# )


# batch_size = 64
# epochs = 18

# history = model.fit(
#     x_train,
#     y_train,
#     validation_split=0.2,
#     batch_size=batch_size,
#     epochs=epochs,
#     verbose=0
# )

In [126]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] < 80:
            self.model.stop_training = True

In [127]:
# Neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=x_train.shape[1]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# # Create an Adam optimizer with a specific learning rate
decay_lr = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps=1000,
    decay_rate=0.88 ,
    staircase=True
)
initial_learning_rate = 0.01
decay_steps = 1000
end_learning_rate = 1e-5
power = 1.0

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate,
    decay_steps,
    end_learning_rate,
    power
)
# optm = tf.keras.optimizers.Adam(learning_rate=decay_lr)
# optm = tf.keras.optimizers.Adam(learning_rate=0.01)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
# Compile the model using the specified optimizer
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Define an early stopping callback
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',  # Monitor validation loss
#     patience=5,          # Number of epochs with no improvement after which training will be stopped
#     restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored metric
# )

# train the model
# model.fit(x_train, y_train, epochs=100, batch_size=500, validation_data=(x_test, y_test), callbacks=[early_stopping])
batch_size = 128
epochs = 100
custom_early_stopping = CustomEarlyStopping()

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    # callbacks=[custom_early_stopping]
    # verbose=0
)
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Evaluate the model on the test set
test_loss = model.evaluate(x_test, y_test)
print(f"Loss: {test_loss:.5f}")#, Accuracy: {test_acc*100:.2f}%")

y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"MSE: {mse:.5f}")
print(f"MAE: {mae:.5f}")

Epoch 1/100
41/41 [==============================] - 1s 7ms/step - loss: 6045099.5000 - val_loss: 691296.6875
Epoch 2/100
41/41 [==============================] - 0s 4ms/step - loss: 1511962.1250 - val_loss: 52609.9141
Epoch 3/100
41/41 [==============================] - 0s 4ms/step - loss: 757321.3750 - val_loss: 21188.0039
Epoch 4/100
41/41 [==============================] - 0s 4ms/step - loss: 604378.9375 - val_loss: 160710.3281
Epoch 5/100
41/41 [==============================] - 0s 3ms/step - loss: 467321.7812 - val_loss: 20080.8457
Epoch 6/100
41/41 [==============================] - 0s 3ms/step - loss: 319290.6250 - val_loss: 22361.1465
Epoch 7/100
41/41 [==============================] - 0s 3ms/step - loss: 333905.6250 - val_loss: 292678.3125
Epoch 8/100
41/41 [==============================] - 0s 3ms/step - loss: 316109.0312 - val_loss: 62546.2734
Epoch 9/100
41/41 [==============================] - 0s 4ms/step - loss: 218300.7031 - val_loss: 785319.6875
Epoch 10/100
41/41 [==